# Prompt pipelining | プロンプトパイプラインプロンプトパイプライニングの背後にあるアイデアは、異なるプロンプトの部品を組み合わせるためのユーザーフレンドリーなインターフェースを提供することです。これは、文字列プロンプトまたはチャットプロンプトを使用して行うことができます。この方法でプロンプトを構築することで、コンポーネントを簡単に再利用できます。> The idea behind prompt pipelining is to provide a user friendly interface for composing different parts of prompts together. You can do this with either string prompts or chat prompts. Constructing prompts this way allows for easy reuse of components.

## String prompt pipelining | 文字列プロンプトのパイプライニング文字列プロンプトを使用する場合、各テンプレートは結合されます。プロンプトを直接扱うことも、文字列（リストの最初の要素はプロンプトでなければなりません）を扱うこともできます。> When working with string prompts, each template is joined together. You can work with either prompts directly or strings (the first element in the list needs to be a prompt).

In [ ]:
from langchain.prompts import PromptTemplate

In [2]:
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)

In [3]:
prompt

PromptTemplate(input_variables=['language', 'topic'], output_parser=None, partial_variables={}, template='Tell me a joke about {topic}, make it funny\n\nand in {language}', template_format='f-string', validate_template=True)

In [4]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny\n\nand in spanish'

以前と同様に、LLMChainでの使用も可能です。> You can also use it in an LLMChain, just like before.

In [5]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [6]:
model = ChatOpenAI()

In [7]:
chain = LLMChain(llm=model, prompt=prompt)

In [8]:
chain.run(topic="sports", language="spanish")

'¿Por qué el futbolista llevaba un paraguas al partido?\n\nPorque pronosticaban lluvia de goles.'

## Chat prompt pipelining | チャットプロンプトのパイプライン化

チャットプロンプトは、メッセージのリストで構成されています。開発者の体験をより良くするため、これらのプロンプトを簡単に作成できる便利な方法を追加しました。このパイプラインにおいて、新たに追加される各要素は、最終的なプロンプト内で新しいメッセージとなります。> A chat prompt is made up a of a list of messages. Purely for developer experience, we've added a convinient way to create these prompts. In this pipeline, each new element is a new message in the final prompt.

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

まず、システムメッセージを用いて基本のChatPromptTemplateを初期化しましょう。必ずしもシステムから始める必要はありませんが、一般的には良い慣習とされています。> First, let's initialize the base ChatPromptTemplate with a system message. It doesn't have to start with a system, but it's often good practice

In [2]:
prompt = SystemMessage(content="You are a nice pirate")

その後、他のメッセージ*または*メッセージテンプレートと組み合わせて簡単にパイプラインを作成できます。変数をフォーマットする必要がない場合は`Message`を使用し、変数をフォーマットする必要がある場合は`MessageTemplate`を使用します。また、単なる文字列を使用することもできます（注：これは自動的にHumanMessagePromptTemplateとして解釈されます）。> You can then easily create a pipeline combining it with other messages *or* message templates.> Use a `Message` when there is no variables to be formatted, use a `MessageTemplate` when there are variables to be formatted. You can also use just a string (note: this will automatically get inferred as a HumanMessagePromptTemplate.)

In [3]:
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)

内部的には、ChatPromptTemplateクラスのインスタンスを作成しているので、以前と同じように使用できます！> Under the hood, this creates an instance of the ChatPromptTemplate class, so you can use it just as you did before!

In [4]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate', additional_kwargs={}),
 HumanMessage(content='hi', additional_kwargs={}, example=False),
 AIMessage(content='what?', additional_kwargs={}, example=False),
 HumanMessage(content='i said hi', additional_kwargs={}, example=False)]

以前と同様に、LLMChainでの使用も可能です。> You can also use it in an LLMChain, just like before.

In [5]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

In [6]:
model = ChatOpenAI()

In [7]:
chain = LLMChain(llm=model, prompt=new_prompt)

In [8]:
chain.run("i said hi")

'Oh, hello! How can I assist you today?'